In [ ]:
# import and setup gemini
import json
from dotenv import load_dotenv
import os
from google import genai

import dataset_manager

# api key must be stored inside a file called .env with the following contents: GEMINI_API_KEY=<your_key_here>
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
# load the dataset from a json file
dataset = dataset_manager.load_dataset("dataset_iter1.json")

In [ ]:
# reload incase the PROMPT was modified or anything else in dataset_manager.py
import importlib
importlib.reload(dataset_manager)

for entry in dataset:
    # format prompt to include the equivalent ltl formulas and already generated translations for reference 
    prompt = dataset_manager.create_prompt(entry)

    # structure output as a list of strings in the json format
    response = client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents = prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': list[str],
        },
    )

    generated_data = json.loads(response.text)
    entry["nlp"] += generated_data

    print(response.text)
print("Done generating data")

In [ ]:
dataset_manager.save_dataset(dataset, "dataset_iter2.json")